# 原生态的XGBoost的使用形式

In [9]:
# 从sklearn 调入所需要的包
from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分割出训练集和验证集
import xgboost as xgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score,recall_score
#导入鸢尾花数据
iris=datasets.load_iris()
data=iris.data
label=iris.target
print(pd.DataFrame(data).head())
print(pd.DataFrame(label).head())
data1=pd.DataFrame(data)
##花蕊长宽 花瓣长宽
data1.columns=['sepal_l','sepal_w','petal_l','petal_w']
print(data1.head())
label1=pd.DataFrame(label)
label1.columns=['label']
print(label1.head())
#注意哦 这里data 和label顺序是一致的千万别打乱

     0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
   0
0  0
1  0
2  0
3  0
4  0
   sepal_l  sepal_w  petal_l  petal_w
0      5.1      3.5      1.4      0.2
1      4.9      3.0      1.4      0.2
2      4.7      3.2      1.3      0.2
3      4.6      3.1      1.5      0.2
4      5.0      3.6      1.4      0.2
   label
0      0
1      0
2      0
3      0
4      0


In [3]:
!pip install xgboost

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [11]:
# 划分训练集和测试环集
train_x,test_x,train_y,test_y=train_test_split(data1.values,label1.values,test_size=0.3,random_state=42)
print("训练集长度：",len(train_x))
print("测试集长度：",len(test_x))

训练集长度： 105
测试集长度： 45


In [17]:
## 转换为DMatrix数据格式
# dtrain=xgb.DMatrix(train_x,label=train_y)
test_data=xgb.DMatrix(test_x,label=test_y)
#设置参数
### multi softmax是使用softmax 产生的分类结果，而multi ：softprob 是输出概率矩阵
xgb_params={
    'eta':0.3,#学习率
    'silent':True,# 输出运行讯息
    'objective':'multi:softprob',#使用多分类生成概率矩阵格式‘multi：softmax’‘multi：softprob’
    'num_class':3,#共有多少类别
    'max_depth':3,# 深度
}
num_round=20 # 跑的步数
# 模型训练
model =xgb.train(xgb_params,xgb.DMatrix(train_x,label=train_y),num_round)
#模型预测
test_pre=model.predict(test_data)
print(test_pre[:5])
#选择出概率最高的列
test_pre_1=np.asarray([np.argmax(row) for row in test_pre])
print('test的预测结果：',test_pre_1)
#模型评估
print('验证集精准率：',precision_score(test_y,test_pre_1,average='macro'))
print('验证集召回率：',recall_score(test_y,test_pre_1,average='macro'))

[[0.00650662 0.96226966 0.03122366]
 [0.9706414  0.02533223 0.00402638]
 [0.0033913  0.0069211  0.9896876 ]
 [0.00654367 0.967749   0.02570731]
 [0.00615655 0.91049767 0.08334578]]
test的预测结果： [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集精准率： 1.0
验证集召回率： 1.0


# Sklearn接口形式使用XgBoost

In [18]:
from xgboost import XGBClassifier

In [21]:
model=XGBClassifier(
     learning_rate=0.01,# 学习lv
    n_estimators=3000,# 步长
    max_depth=4,#深度
    objective='binary:logistic',
    seed=27
)
model.fit(train_x,train_y)
#预测
#输出预测结果
test_pre2=model.predict(test_x)
print(test_pre2)
#模型评估
print('验证集精准率：',precision_score(test_y,test_pre2,average='macro'))
print('验证集召回率',recall_score(test_y,test_pre2,average='macro'))

G:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
G:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集精准率： 1.0
验证集召回率 1.0


G:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
